In [15]:
import logging
import os, sys
from pathlib import Path
import numpy as np

verbose = True

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import cv2


__file__ = os.path.abspath('helper_precomputing_CNN_features.ipynb')
DNN_lib_path = Path(__file__).parents[1].__str__()

# from contextlib import redirect_stdout
sys.path.append('..')

#from keras.preprocessing.image import img_to_array

# setup paths

path_data = '/media/tiesbarendse/DATA/be_ts_large/'
path_cache = '/media/tiesbarendse/DATA/be_ts_large_cache/'

# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s, [%(levelname)-8s] [%(filename)s:%(lineno)d] %(message)s',
    handlers=[
        logging.FileHandler(f"{DNN_lib_path}/logs.log"),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()

resnet_len_features = 2048
resnet50_model_size = (32, 32)


custom_model_name = 'ResNet50_pop_grids_large'
pretrained_model_name = 'ResNet50'
pooling = 'max'
""" 
Save pretrained features array computed over all frames of each video 
using given pretrained model and pooling method

:pretrained_model_name: pretrained model object loaded using `load_pretrained_model`
:pooling: pooling method used with pretrained model
:model_weights_path: path to custom model weights if we want to load CNN model we've fine-tuned to produce features (e.g. for LRCNN)
:custom_model_name: custom output name to append to pretrained model name

"""

# setup path to save features
path_features = path_cache + 'features/' + custom_model_name

if not os.path.exists(path_features):
    os.mkdir(path_features)

model = ResNet50(include_top=False, weights=None,
                 pooling=pooling, input_shape=(32, 32, 3))

bes_filenames = [be_filename for be_filename in os.listdir(path_data) if be_filename[:5] == 'trajs']
k = len(bes_filenames)

try:
    for c, be_filename in enumerate(bes_filenames):
        # build output path
        be_name = be_filename[:-4]
        path_output = f"{path_features}/{be_name}"
        path_frames = f"{path_data}/{be_name}"

        if os.path.exists(path_output + '.npy'):
            if verbose:
                logger.info("Features already cached: {}.npy".format(path_output))
            continue

        be = np.load(f'{path_data}/{be_filename}')

        # initialize features list
        features = []

        # load each frame in vid and get features
        for j, frame in enumerate(be):
            frame[np.isnan(frame)] = 0
            frame = np.array([frame, frame, frame])
            frame = np.resize(frame, new_shape=(*resnet50_model_size, 3))
            frame = np.expand_dims(frame, axis=0)

            # get features from pretrained model
            feature = model.predict(frame).ravel()
            features.append(feature)
        
        # convert to arrays
        features = np.array(features)

        np.save(f'{path_features}/{be_filename[:-4]}_f.npy', features)
        print(f"\rLoaded \"{be_name:30s}\" ({c}/{k})")

except Exception as e:
    raise TimeoutError(f'Could not finish. Timed out at {c}/{k} boarding event features.')
print(f'=============================\n Finished. Saved {c} boarding event features in {path_data}.\n=============================')

In [11]:
[be_filename for be_filename in os.listdir(path_data) if be_filename[:5] == 'trajs']

['trajs_2017-03-01_Ut_3020_door_3.npy',
 'trajs_2017-03-01_Ut_3024_door_3.npy',
 'trajs_2017-03-01_Ut_3026_door_3.npy',
 'trajs_2017-03-01_Ut_3028_door_4.npy',
 'trajs_2017-03-01_Ut_3034_door_4.npy',
 'trajs_2017-03-01_Ut_3040_door_4.npy',
 'trajs_2017-03-01_Ut_3048_door_4.npy',
 'trajs_2017-03-01_Ut_3118_door_3.npy',
 'trajs_2017-03-01_Ut_3154_door_3.npy',
 'trajs_2017-03-02_Ut_3042_door_4.npy',
 'trajs_2017-03-02_Ut_3044_door_4.npy',
 'trajs_2017-03-02_Ut_3048_door_3.npy',
 'trajs_2017-03-02_Ut_3050_door_3.npy',
 'trajs_2017-03-02_Ut_3056_door_3.npy',
 'trajs_2018-01-04_Ut_3164_door_4.npy',
 'trajs_2018-01-04_Ut_3172_door_4.npy',
 'trajs_2018-01-04_Ut_3536_door_4.npy',
 'trajs_2018-01-05_Ut_3020_door_3.npy',
 'trajs_2018-01-05_Ut_3032_door_4.npy',
 'trajs_2018-01-05_Ut_3036_door_4.npy',
 'trajs_2018-01-05_Ut_3046_door_3.npy',
 'trajs_2018-01-05_Ut_3050_door_3.npy',
 'trajs_2018-01-05_Ut_3056_door_3.npy',
 'trajs_2018-01-05_Ut_3062_door_3.npy',
 'trajs_2018-01-05_Ut_3066_door_3.npy',


In [10]:
os.listdir(path_data)

['labels.csv',
 'trajs_2017-03-01_Ut_3020_door_3.npy',
 'trajs_2017-03-01_Ut_3024_door_3.npy',
 'trajs_2017-03-01_Ut_3026_door_3.npy',
 'trajs_2017-03-01_Ut_3028_door_4.npy',
 'trajs_2017-03-01_Ut_3034_door_4.npy',
 'trajs_2017-03-01_Ut_3040_door_4.npy',
 'trajs_2017-03-01_Ut_3048_door_4.npy',
 'trajs_2017-03-01_Ut_3118_door_3.npy',
 'trajs_2017-03-01_Ut_3154_door_3.npy',
 'trajs_2017-03-02_Ut_3042_door_4.npy',
 'trajs_2017-03-02_Ut_3044_door_4.npy',
 'trajs_2017-03-02_Ut_3048_door_3.npy',
 'trajs_2017-03-02_Ut_3050_door_3.npy',
 'trajs_2017-03-02_Ut_3056_door_3.npy',
 'trajs_2018-01-04_Ut_3164_door_4.npy',
 'trajs_2018-01-04_Ut_3172_door_4.npy',
 'trajs_2018-01-04_Ut_3536_door_4.npy',
 'trajs_2018-01-05_Ut_3020_door_3.npy',
 'trajs_2018-01-05_Ut_3032_door_4.npy',
 'trajs_2018-01-05_Ut_3036_door_4.npy',
 'trajs_2018-01-05_Ut_3046_door_3.npy',
 'trajs_2018-01-05_Ut_3050_door_3.npy',
 'trajs_2018-01-05_Ut_3056_door_3.npy',
 'trajs_2018-01-05_Ut_3062_door_3.npy',
 'trajs_2018-01-05_Ut_306

In [5]:
bes_filenames

[]

In [ ]:
frame_paths = [f"{path_frames}/{frame}" for frame in os.listdir(path_frames)]
frames = [np.load(path_frame) for path_frame in frame_paths]

In [ ]:
frames[0]

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])